# Question 1

Use the Decision Tree model trained on the UCI-HAR dataset to predict the activities that you performed. Report the accuracy, precision, recall and confusion matrix of the model. You have three version of UCI dataset you can use a)Raw data from accelerometer, b)TSFEL featurised data, c)Features provided by author. Choose which version to use, ensuring that your test data is similar to your training data. How did the model perform? [1 marks]


In [232]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tsfel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

First, let's train the TSFEL model on the given dataset.


In [233]:
time = 10 # Setting the time window for each sample
offset = 100 # Skipping the first 100 rows to remove noise
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

combined_dir = os.path.join("../Combined")

X_train=[]
y_train=[]
dataset_dir = os.path.join(combined_dir,"Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))

    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test=[]
y_test=[]
dataset_dir = os.path.join(combined_dir,"Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))
    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_test.append(df.values)
        y_test.append(classes[folder])

X_test = np.array(X_test)
y_test = np.array(y_test)

# USE THE BELOW GIVEN DATA FOR TRAINING and TESTING purposes

# concatenate the training and testing data
X = np.concatenate((X_train,X_test))
y = np.concatenate((y_train,y_test))

# split the data into training and testing sets. Change the seed value to obtain different random splits.
seed = 4
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=seed,stratify=y)

print("Training data shape: ", X_train.shape) # (number of samples, number of time steps (x (=10) sec * 50Hz), number of features)
print("Testing data shape: ", X_test.shape)
print("Training labels shape: ", y_train.shape)
print("Testing labels shape: ", y_test.shape)

# Data - 3D array of 180 samples
# Splitting 180 samples into 70% training and 30% testing data so 126 samples for training and 54 samples for testing
# 126 / 6 = 21 meaning 21 samples per activity
# 500 = 10 * 50 meaning 500 time steps per sample as 10 seconds of data is collected at 50Hz
# 3 signifies the number of features (x, y, z) for each time step

# Labels - 1D array of 180 labels
# 126 means 126 labels for the training data according to the corresponding activity
# 54 means 54 labels for the testing data according to the corresponding activity

Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)
Training labels shape:  (126,)
Testing labels shape:  (54,)


In [234]:
# the TSFEL library expects the data in pandas dataframe format where each column represents a time series and each row represents a time step
X_train_dfs = []

for sample in X_train:
  df = pd.DataFrame(sample, columns=["accx", "accy", "accz"])
  X_train_dfs.append(df)

X_test_dfs = []

for sample in X_test:
  df = pd.DataFrame(sample, columns=["accx", "accy", "accz"])
  X_test_dfs.append(df)

# we will be using the default tsfel configuration for the features
# this is a dictionary that specifies a wide range of features to be extracted across different domains (statistical, temporal, spectral, etc.).
tsfel_config = tsfel.get_features_by_domain()
# print(len(tsfel_config["spectral"]) + len(tsfel_config["fractal"]) + len(tsfel_config["statistical"]) + len(tsfel_config["temporal"]))
# ? This gives 67

# extract features for X_train
# iterating over each sample to characterize its unique features
# if we combine all our samples into a single dataframe and then extract features, TSFEL might interpret the entire concatenated time series as a single entity. This could lead to features being calculated across different activities, which might not be meaningful or useful for our classification task.
# since our accelerometer data is sampled at 50Hz, we need to tell TSFEL that the sampling frequency is 50Hz so that when it calculates features that depend on time, it can use the correct time intervals.
# TSFEL uses FFT to calculate the spectral features and hence requires the sampling frequency to be the same as the actual sampling frequency of the data.
# we haven't mentioned overlap and window_size as the data which we are using is already windowed and has no overlap.
# according to the TSFEL documentation, if we have access to the actual sampling frequency, we should provide it to the library as a good practice.
X_train_tsfel_dfs = []
for df in X_train_dfs:
    feature_df = tsfel.time_series_features_extractor(tsfel_config, df, fs=50)
    X_train_tsfel_dfs.append(feature_df)
# the output of tsfel.time_series_features_extractor(tsfel_config, df, fs=50) is a pandas dataframe containing the extracted features where each row represents a sample and each column represents a feature as specified in the tsfel_config dictionary.
# ? Here each dataframe is of size 1 x (3 * 384) = 1 x 1152

# convert the list of feature dataframe back to a numpy array
# since we have 126 samples and each sample has 1152 features, the shape of the array should be (126, 1152)
# there are some NaN values because some features are not applicable to the data and hence are not calculated
# fill NaN values with 0 to ensure the array is compatible with scikit-learn
X_train_tsfel = pd.concat(X_train_tsfel_dfs, axis=0).fillna(0).values

# Same for X_test
X_test_tsfel_dfs = []
for df in X_test_dfs:
    feature_df = tsfel.time_series_features_extractor(tsfel_config, df, fs=50)
    X_test_tsfel_dfs.append(feature_df)
# Since we have 54 samples and each sample has 1152 features, the shape of the array should be (54, 1152)
X_test_tsfel = pd.concat(X_test_tsfel_dfs, axis=0).fillna(0).values

*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***


In [235]:
dtc_tsfel = DecisionTreeClassifier()
dtc_tsfel.fit(X_train_tsfel, y_train)
y_pred_tsfel = dtc_tsfel.predict(X_test_tsfel)
acc_tsfel = accuracy_score(y_test, y_pred_tsfel)

# Since, this model is using the provided dataset which is balanced, we will use macro average for precision and recall
prec_tsfel = precision_score(y_test, y_pred_tsfel, average='macro')
rec_tsfel = recall_score(y_test, y_pred_tsfel, average='macro')

conf_mx_tsfel = confusion_matrix(y_test, y_pred_tsfel)

print("TSFEL Accuracy:", acc_tsfel)
print("TSFEL Precision:", prec_tsfel)
print("TSFEL Recall:", rec_tsfel)
print()
print("TSFEL Confusion Matrix:")
print(conf_mx_tsfel)

TSFEL Accuracy: 0.8518518518518519
TSFEL Precision: 0.8655303030303031
TSFEL Recall: 0.8518518518518517

TSFEL Confusion Matrix:
[[9 0 0 0 0 0]
 [2 6 1 0 0 0]
 [1 1 7 0 0 0]
 [0 1 0 6 2 0]
 [0 0 0 0 9 0]
 [0 0 0 0 0 9]]


In [272]:
offset = 125

custom_data_dir = os.path.join("../Krustykrabs")

X_custom_data=[]
y_custom_data=[]

for folder in folders:
    files = os.listdir(os.path.join(custom_data_dir,folder))

    for file in files:
        if file == ".DS_Store":
            continue

        df = pd.read_csv(os.path.join(custom_data_dir,folder,file),sep=",",header=0)
        df.drop(["time", "TgF"], axis=1, inplace=True)

        df = df[offset:offset+time*50]

        for col in df.columns:
            df[col] = (2 * (df[col] - df[col].min()) / (df[col].max() - df[col].min())) - 1

        X_custom_data.append(df.values)
        y_custom_data.append(classes[folder])


seed = 4
X_train_custom_data, X_test_custom_data, y_train_custom_data, y_test_custom_data = train_test_split(X_custom_data,y_custom_data,test_size=0.3,random_state=seed,stratify=y_custom_data)

In [273]:
X_train_dfs_custom_data = []

for sample in X_train_custom_data:
  df = pd.DataFrame(sample)
  X_train_dfs_custom_data.append(df)

X_test_dfs_custom_data = []

for sample in X_test_custom_data:
  df = pd.DataFrame(sample)
  X_test_dfs_custom_data.append(df)

tsfel_config_custom_data = tsfel.get_features_by_domain()

X_train_tsfel_dfs_custom_data = []
for df in X_train_dfs_custom_data:
    feature_df = tsfel.time_series_features_extractor(tsfel_config_custom_data, df, fs=50)
    X_train_tsfel_dfs_custom_data.append(feature_df)

X_train_tsfel_custom_data = pd.concat(X_train_tsfel_dfs_custom_data, axis=0).fillna(0).values

X_test_tsfel_dfs_custom_data = []
for df in X_test_dfs_custom_data:
    feature_df = tsfel.time_series_features_extractor(tsfel_config_custom_data, df, fs=50)
    X_test_tsfel_dfs_custom_data.append(feature_df)

X_test_tsfel_custom_data = pd.concat(X_test_tsfel_dfs_custom_data, axis=0).fillna(0).values

*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***


In [275]:
y_pred_tsfel_custom_data = dtc_tsfel.predict(X_test_tsfel_custom_data)
acc_tsfel_custom_data = accuracy_score(y_test_custom_data, y_pred_tsfel_custom_data)

# Since, this model is using the provided dataset which is balanced, we will use macro average for precision and recall
prec_tsfel_custom_data = precision_score(y_test_custom_data, y_pred_tsfel_custom_data, average='macro')
rec_tsfel_custom_data = recall_score(y_test_custom_data, y_pred_tsfel_custom_data, average='macro')

conf_mx_tsfel_custom_data = confusion_matrix(y_test_custom_data, y_pred_tsfel_custom_data)

print("TSFEL Accuracy:", acc_tsfel_custom_data)
print("TSFEL Precision:", prec_tsfel_custom_data)
print("TSFEL Recall:", rec_tsfel_custom_data)
print()
print("TSFEL Confusion Matrix:")
print(conf_mx_tsfel_custom_data)

TSFEL Accuracy: 0.2222222222222222
TSFEL Precision: 0.047619047619047616
TSFEL Recall: 0.16666666666666666

TSFEL Confusion Matrix:
[[2 0 0 0 0 0]
 [2 0 0 0 0 0]
 [2 0 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]]


/Users/mohitpanchal/Academics/Semester-3/ES335-ML/es335-24-fall-assignment-1/venv_es335_1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [278]:
X_test_tsfel_custom_data.shape

(9, 1152)

# Question 2

Use the data you collected to predict the activities that you performed. Decide whether to apply preprocessing and featurization, and if so, choose the appropriate methods. How did the model perform? [1 marks]


In [310]:
dtc_custom_data = DecisionTreeClassifier()
dtc_custom_data.fit(X_train_tsfel_custom_data, y_train_custom_data)
y_pred_tsfel_custom_data = dtc_custom_data.predict(X_test_tsfel_custom_data)
acc_tsfel_custom_data = accuracy_score(y_test_custom_data, y_pred_tsfel_custom_data)

# Since, this model is using the provided dataset which is balanced, we will use macro average for precision and recall
prec_tsfel_custom_data = precision_score(y_test_custom_data, y_pred_tsfel_custom_data, average='macro')
rec_tsfel_custom_data = recall_score(y_test_custom_data, y_pred_tsfel_custom_data, average='macro')

conf_mx_tsfel_custom_data = confusion_matrix(y_test_custom_data, y_pred_tsfel_custom_data)

print("TSFEL Accuracy:", acc_tsfel_custom_data)
print("TSFEL Precision:", prec_tsfel_custom_data)
print("TSFEL Recall:", rec_tsfel_custom_data)
print()
print("TSFEL Confusion Matrix:")
print(conf_mx_tsfel_custom_data)

TSFEL Accuracy: 0.8888888888888888
TSFEL Precision: 0.9444444444444443
TSFEL Recall: 0.9166666666666666

TSFEL Confusion Matrix:
[[1 0 1 0 0 0]
 [0 2 0 0 0 0]
 [0 0 2 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
